## Install packages

In [1]:
%%capture
!pip -q install accelerate transformers einops peft bitsandbytes --upgrade
!pip install vllm
# !pip install -U ipywidgets
# !pip install datasets==2.17 --force-reinstall
# ERROR: Cannot uninstall multidict 6.0.4, RECORD file not found. 
# You might be able to recover from this via: 'pip install --force-reinstall --no-deps multidict==6.0.4'.

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset, Dataset
from transformers import DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os
import json

In [4]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

## Load model

In [6]:
MODEL_NAME = "qwen-1.5-4B-lora_sft"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained("/home/siyu/Documents/6103_group/4b_lora_ckpt_1epoch", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/qwen1-5-4b-ckpt/checkpoint-1epoch", trust_remote_code=True)
print(tokenizer.chat_template)
print("\n\n")
# template = tokenizer.default_chat_template
# template = template.replace("role", "from").replace("content", "value")  # Change the system token
# tokenizer.chat_template = template
# print(tokenizer.chat_template)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{% set system_message = 'You are a helpful assistant.' %}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|im_start|>system\n' + system_message + '<|im_end|>\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\n' + content + '<|im_end|>\n<|im_start|>assistant\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\n' }}{% endif %}{% endfor %}





In [8]:
lora_config = PeftConfig.from_pretrained("/home/siyu/Documents/6103_group/4b_lora_ckpt_1epoch")
print(lora_config)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-4B-Chat",
    torch_dtype="auto",
    device_map="auto"
)

model = PeftModel.from_pretrained(model, 
                                  model_id="/home/siyu/Documents/6103_group/4b_lora_ckpt_1epoch", 
                                  #config = lora_config,
                                  trust_remote_code=True)


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='../model/Qwen1.5-4B', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=128, target_modules={'q_proj', 'o_proj', 'gate_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj'}, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


KeyboardInterrupt: 

In [6]:
# print(model)

# print(model.generation_config)

## Load & preprocess test data

In [7]:
role_mapping = {"human":"user", "gpt":"assistant"}
def convert_data(example: list, role_mapping = role_mapping) -> list:
    res = []
    for text in example["conversations"]:
        converted_data = {}
        converted_data['role'] = role_mapping[text['from']]
        converted_data['content'] = text['value']
        res.append(converted_data)
    return res
    

In [12]:
DATA_NAME = "NER_literature"

with open(os.path.join("/home/siyu/Documents/6103_group/test_data", f"mit-movie.json"), "r") as f:
    examples = json.load(f)
print(tokenizer.apply_chat_template(convert_data(examples[0]), tokenize=False))

converted_examples =  [convert_data(chat) for chat in examples]
golds = [example[-1]['content'] for example in converted_examples]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Text: are there any good romantic comedies out right now<|im_end|>
<|im_start|>assistant
I've read this text.<|im_end|>
<|im_start|>user
What describes character in the text?<|im_end|>
<|im_start|>assistant
[]<|im_end|>



In [15]:
def format_input(chat):
    formatted_chat = tokenizer.apply_chat_template(chat, tokenize=False)
    input_chat = formatted_chat.split("assistant")
    input_chat[-1] = "\n"
    input_chat = "assistant".join(input_chat)
    return input_chat

def get_response(responses):
    responses = [r.split('assistant')[-1].strip().rstrip('<|im_end|>') for r in responses]
    return responses

## Generate responses

In [ ]:
from tqdm.auto import tqdm
outputs = []
for i in tqdm(range(len(converted_examples))):
    chat = converted_examples[i]
    inputs = tokenizer(format_input(chat), return_tensors="pt", return_attention_mask=False).to(device)
    output = model.generate(**inputs, max_length=384)
    output_text = tokenizer.batch_decode(output)[0]
    outputs.append(output_text)


  0%|          | 0/29304 [00:00<?, ?it/s]

In [ ]:
responses = get_response(outputs)

with open(f"./working/{DATA_NAME}_responses.json", "w") as json_file:
    json.dump(responses, json_file, ensure_ascii=False)

with open(f"./working/{DATA_NAME}_outputs_raw.json", "w") as json_file:
    json.dump(outputs, json_file, ensure_ascii=False)

In [ ]:
# print(golds[:100])
# print(responses[:100])

## NER evaluator

In [ ]:
import re
import string

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))


def parser(text):
    try:
        match = re.match(r'\[(.*?)\]', text)
        if match:
            text = match.group()
        else:
            text = '[]'
        items = json.loads(text)
        formatted_items = []
        for item in items:
            if isinstance(item, list) or isinstance(item, tuple):
                item = tuple([normalize_answer(element) for element in item])
            else:
                item = normalize_answer(item)
            if item not in formatted_items:
                formatted_items.append(item)
        return formatted_items
    except Exception:
        return []

class NEREvaluator:
    def evaluate(self, preds: list, golds: list):
        n_correct, n_pos_gold, n_pos_pred = 0, 0, 0
        for pred, gold in zip(preds, golds):
            gold_tuples = parser(gold)
            pred_tuples = parser(pred)
            for t in pred_tuples:
                if t in gold_tuples:
                    n_correct += 1
                n_pos_pred += 1
            n_pos_gold += len(gold_tuples)
        prec = n_correct / (n_pos_pred + 1e-10)
        recall = n_correct / (n_pos_gold + 1e-10)
        f1 = 2 * prec * recall / (prec + recall + 1e-10)
        return {
            'precision': prec,
            'recall': recall,
            'f1': f1,
        }


In [ ]:
eval_result = NEREvaluator().evaluate(responses, golds)
print(f'Precision: {eval_result["precision"]}, Recall: {eval_result["recall"]}, F1: {eval_result["f1"]}')

eval_result['data_name'] = DATA_NAME
eval_result['model_name'] = MODEL_NAME
with open(f"./working/{DATA_NAME}_eval_results.json", "w+") as json_file:
    json.dump(eval_result, json_file, ensure_ascii=False, indent=4)